In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45640
2,Huelva,Confirmados PDIA 14 días,296
3,Huelva,Tasa PDIA 14 días,"57,680690609349725"
4,Huelva,Confirmados PDIA 7 días,159
5,Huelva,Tasa PDIA 7 dias,"30,983884482725024"
6,Huelva,Total Confirmados,45842
7,Huelva,Curados,44687
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45640.0


/tmp/ipykernel_40039/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12577.0


/tmp/ipykernel_40039/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_40039/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_40039/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_40039/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 8990 personas en los últimos 7 días 

Un positivo PCR cada 3345 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45640.0,159.0,296.0,513170.0,30.983884,57.680691,61.0
Huelva-Costa,27008.0,82.0,161.0,289548.0,28.320002,55.603907,31.0
Condado-Campiña,14181.0,61.0,108.0,156231.0,39.044748,69.128406,24.0
Cartaya,2026.0,36.0,57.0,20083.0,179.256087,283.822138,13.0
Huelva (capital),12577.0,16.0,43.0,143837.0,11.123703,29.894951,10.0
Moguer,1979.0,14.0,26.0,21867.0,64.023414,118.900627,6.0
Sierra de Huelva-Andévalo Central,4025.0,15.0,26.0,67391.0,22.258165,38.580819,5.0
Valverde del Camino,853.0,9.0,16.0,12750.0,70.588235,125.490196,4.0
Palos de la Frontera,1041.0,11.0,14.0,11742.0,93.680804,119.230114,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,80.0,4.0,6.0,1697.0,235.710077,353.565115,0.0
Villarrasa,227.0,3.0,7.0,2211.0,135.685210,316.598824,2.0
Cartaya,2026.0,36.0,57.0,20083.0,179.256087,283.822138,13.0
Manzanilla,141.0,2.0,3.0,2118.0,94.428706,141.643059,0.0
Valverde del Camino,853.0,9.0,16.0,12750.0,70.588235,125.490196,4.0
Palos de la Frontera,1041.0,11.0,14.0,11742.0,93.680804,119.230114,4.0
Moguer,1979.0,14.0,26.0,21867.0,64.023414,118.900627,6.0
Bonares,464.0,4.0,6.0,6060.0,66.006601,99.009901,1.0
Villalba del Alcor,500.0,3.0,3.0,3366.0,89.126560,89.126560,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lepe,3026.0,6.0,19.0,27880.0,21.520803,68.149211,2.0,0.315789
Bollullos Par del Condado,1289.0,3.0,9.0,14387.0,20.852158,62.556475,0.0,0.333333
Gibraleón,947.0,4.0,11.0,12737.0,31.404569,86.362566,0.0,0.363636
Huelva (capital),12577.0,16.0,43.0,143837.0,11.123703,29.894951,10.0,0.372093
Ayamonte,2087.0,7.0,17.0,21104.0,33.169067,80.553450,0.0,0.411765
Villarrasa,227.0,3.0,7.0,2211.0,135.685210,316.598824,2.0,0.428571
Rociana del Condado,668.0,1.0,2.0,7939.0,12.596045,25.192090,0.0,0.500000
Isla Cristina,3133.0,5.0,10.0,21393.0,23.372131,46.744262,2.0,0.500000
Huelva-Costa,27008.0,82.0,161.0,289548.0,28.320002,55.603907,31.0,0.509317
